In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shayanfazeli_heartbeat_path = kagglehub.dataset_download('shayanfazeli/heartbeat')

print('Data source import complete.')


100%|██████████| 98.8M/98.8M [00:32<00:00, 3.19MB/s]

Extracting files...


Data source import complete.


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import plotly.express as px

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Dropout


ModuleNotFoundError: No module named 'imblearn'

In [ ]:
#train_data = pd.read_csv(r'/kaggle/input/heartbeat/mitbih_train.csv',header=None)
train_data = pd.read_csv('dataset/mitbih_train.csv',header=None)
train_data.head()

NameError: name 'pd' is not defined

In [ ]:
#test_data = pd.read_csv(r'/kaggle/input/heartbeat/mitbih_test.csv',header=None)
test_data = pd.read_csv('dataset/mitbih_test.csv',header=None)
test_data.head()

In [ ]:
print(f"train data size: {train_data.shape}")
print(f"test data size: {test_data.shape}")

In [ ]:
labels = {
    0: "Normal",
    1: "Artial Premature",
    2: "Premature ventricular contraction",
    3: "Fusion of ventricular and normal",
    4: "Fusion of paced and normal"
}


value_counts = train_data.iloc[:,-1].value_counts().rename(labels)

pie_fig = px.pie(names=value_counts.index, values=value_counts.values,
                 title="The Percentage of Each Label in The Train Dataset")

pie_fig.update_layout(title_x=0.5, width=800, height=600)
pie_fig.show()


In [ ]:
# See the number of each class in test dataset
value_counts_test = test_data.iloc[:,-1].value_counts().rename(labels)

pie_fig = px.pie(names=value_counts_test.index, values=value_counts_test.values,
                 title="The Percentage of Each Label in The Test Dataset")

pie_fig.update_layout(title_x=0.5, width=800, height=600)
pie_fig.show()

In [ ]:
ecg_0 = train_data.loc[train_data[187]==0, :].iloc[1,:187]
ecg_1 = train_data.loc[train_data[187]==1, :].iloc[1,:187]
ecg_2 = train_data.loc[train_data[187]==2, :].iloc[1,:187]
ecg_3 = train_data.loc[train_data[187]==3, :].iloc[1,:187]
ecg_4 = train_data.loc[train_data[187]==4, :].iloc[1,:187]

ecg_df = pd.DataFrame({'Normal': ecg_0,
                       'Artial Premature': ecg_1,
                       'Premature ventricular contraction': ecg_2,
                       'Fusion of ventricular and normal': ecg_3,
                       'Fusion of paced and normal': ecg_4})
ecg_plot = px.line(ecg_df,
                  labels={'index':'Time Intervals', 'value':'Amplitude (mV)', 'variable':'Categories'},
                  title="One ECG form Each Category")
ecg_plot.update_layout(title_x=0.25, width=1000, height=500)
ecg_plot.show()

In [ ]:
data = train_data.iloc[:,:187]
labels = train_data.iloc[:,187]

In [ ]:
x_train = train_data.iloc[:,:187]
y_train= train_data.iloc[:,187]

x_test = test_data.iloc[:,:187]
y_test = test_data.iloc[: ,187]

In [ ]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)

print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

In [ ]:
model = Sequential([
    Input(shape=(187,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20,
    batch_size=64,
    verbose=1
)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")

y_pred = np.argmax(model.predict(x_test), axis=1)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()